In [57]:
## 1) Preparación de los datos:

In [58]:
#!pip install funpymodeling
!#pip install ydata_profiling

In [59]:
#HABILITO IMPORTACION DESDE DRIVE
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#Importo librerías
from funpymodeling.exploratory import status, freq_tbl
import pandas as pd
import seaborn as sns
import numpy as np
from ydata_profiling import ProfileReport

ModuleNotFoundError: No module named 'ydata_profiling'

In [61]:
all_data= pd.read_csv('/home/chipa/TP8/test.csv', sep = ',')
all_data.head(4)

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied


In [62]:
#Importo data
# all_data = pd.read_csv("test.csv", sep=',', index_col=0)
all_data = pd.read_csv('/home/chipa/TP8/test.csv', sep = ',', index_col=0)
all_data.shape

(25976, 24)

In [63]:
# Este dataset tiene muchas columnas, así que solo nos vamos a quedar con algunas:
data = all_data[['Age', 'Class', 'Inflight wifi service', 'Ease of Online booking', 'Seat comfort', 'Checkin service', 'satisfaction']].copy()

In [64]:
data.head(4)

,Age,Class,Inflight wifi service,Ease of Online booking,Seat comfort,Checkin service,satisfaction
0,52,Eco,5,3,3,2,satisfied
1,36,Business,1,3,5,3,satisfied
2,20,Eco,2,2,2,2,neutral or dissatisfied
3,44,Business,0,0,4,3,satisfied


In [67]:
# Cambiar el nombre de las columnas para evitar espacios en blanco y que sean más concisas.
data.rename(
        columns = {
            'Inflight wifi service':'Wifi',
            'Ease of Online booking':'Booking',
            'Seat comfort':'Seat',
            'Checkin service':'Checkin',
            },
    inplace = True)

In [72]:
data['Checkin'].unique()

array([2, 3, 4, 1, 5])

In [70]:
data['Seat'].unique()

array([3, 5, 2, 4, 1])

In [69]:
data['Wifi'].unique()

array([5, 1, 2, 0, 3, 4])

In [71]:
data['Booking'].unique()

array([3, 2, 0, 4, 5, 1])

In [6]:
data['satisfaction'].value_counts()

satisfaction
neutral or dissatisfied    14573
satisfied                  11403
Name: count, dtype: int64

In [7]:
# Cambiar los valores de la columna `satisfaction` a 1-0:
class_map = {'neutral or dissatisfied':0, 'satisfied':1}
data['satisfaction'] = data['satisfaction'].map(class_map)
data['satisfaction'].value_counts()

satisfaction
0    14573
1    11403
Name: count, dtype: int64

In [ ]:
# ProfileReport(data, minimal=True)
# Age la voy a tener que discretizar...

In [8]:
# Hago la discretización de Age y guardo los bins para usar después:
# No voy a cambiar el nombre de Age esta vez, lo reemplazo de una.
saved_bins_age = pd.qcut(data['Age'], q=5, retbins=True)[1]


In [9]:
# Guardar los bins con pickle
import pickle
with open('saved_bins_age.pickle', 'wb') as handle:
    pickle.dump(saved_bins_age, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
data['Age_disc'] = pd.cut(data['Age'], bins=saved_bins_age, include_lowest=True, duplicates='drop')

In [11]:
# Hacer GET DUMMIES
# Aplicar get_dummies a las columnas discretizadas
dummies_age = pd.get_dummies(data['Age_disc'], prefix='Age')
dummies_class = pd.get_dummies(data['Class'], prefix='Class')

# Concatenar con el dataframe original
data = pd.concat([data, dummies_age, dummies_class], axis=1)

status(data)

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,Age,0,0.0,0,0.000000,75,int64
1,Class,0,0.0,0,0.000000,3,object
2,Wifi,0,0.0,813,0.031298,6,int64
3,Booking,0,0.0,1195,0.046004,6,int64
4,Seat,0,0.0,0,0.000000,5,int64
5,Checkin,0,0.0,0,0.000000,5,int64
6,satisfaction,0,0.0,14573,0.561018,2,int64
7,Age_disc,0,0.0,0,0.000000,5,category
8,"Age_(6.999, 25.0]",0,0.0,20416,0.785956,2,bool
9,"Age_(25.0, 36.0]",0,0.0,20704,0.797043,2,bool


## 2) Clasificación
- Su variable target o de interés a clasificar es `satisfaction`.

- Recuerden comentar y NO utilizar la siguiente celda:
    ```
    data_x = data_x.values
    data_y = data_y.values
    ```

- Utilicen el 30% del dataset para test.

- Para el Random Forest consideren los parámetros `n_estimators = 5000` y `random_state = 19`

In [12]:
# Elimino las variables que no me sirven
df_final = data.drop(['Age', 'Class', 'Age_disc'], axis=1)

In [13]:
data_x = df_final.drop('satisfaction', axis=1)
data_y = df_final['satisfaction']

In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y,random_state=19, test_size=0.3)

In [15]:
# Creo el modelo:
from sklearn.ensemble import RandomForestClassifier

# Creamos 1000 decision trees
rf = RandomForestClassifier(n_estimators = 5000, random_state = 19)

In [16]:
# Entreno el modelo:

rf.fit(x_train, y_train)

,n_estimators,5000
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [17]:
# Guarden el modelo con el nombre rf.pkl.
# Guardar en el disco
filename = 'rf.pkl'
pickle.dump(rf, open(filename, 'wb')) # rf = nuestro modelo

# Guardar el nombre de las columnas
# Guardamos las columnas x (sin satisfaction)
with open('categories_ohe.pickle', 'wb') as handle:
    pickle.dump(data_x.columns, handle, protocol=pickle.HIGHEST_PROTOCOL)

Creación de la interfaz => app.py